In [117]:
import numpy as np
import time
import os as os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, PReLU
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.activations import swish
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split


data_dir = 'Image_classification_data/patch_images'  

dataLabels = pd.read_csv('Image_classification_data/data_labels_mainData.csv')
dataLabels['isCancerous'] = dataLabels['isCancerous'].astype(str)

img_size = (27, 27)
batch_size = 32

train_data, test_data = train_test_split(dataLabels, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=42)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
)

early_stop = EarlyStopping(
    monitor='val_accuraccy',
    patience=15,
    min_delta=0.001,
    restore_best_weights=True
)

isCancerous_class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(dataLabels['isCancerous']),
    y=dataLabels['isCancerous']
)

isCancerous_class_weights_dict = dict(enumerate(isCancerous_class_weights))

cellType_class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(dataLabels['cellTypeName']),
    y=dataLabels['cellTypeName']
)

cellType_class_weights_dict = dict(enumerate(cellType_class_weights))

print(dataLabels['isCancerous'].value_counts())
print(dataLabels['cellTypeName'].value_counts())

isCancerous
0    5817
1    4079
Name: count, dtype: int64
cellTypeName
epithelial      4079
inflammatory    2543
fibroblast      1888
others          1386
Name: count, dtype: int64


In [118]:
isCancerous_train_data = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory='Image_classification_data/patch_images',
    x_col='ImageName', 
    y_col='isCancerous',  
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

isCancerous_val_data = val_datagen.flow_from_dataframe(
    dataframe=val_data,
    directory='Image_classification_data/patch_images',
    x_col='ImageName',  
    y_col='isCancerous',  
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
)

isCancerous_test_data = val_datagen.flow_from_dataframe(
    dataframe=test_data,
    directory='Image_classification_data/patch_images',
    x_col='ImageName',  
    y_col='isCancerous',  
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
)

Found 5937 validated image filenames belonging to 2 classes.
Found 1979 validated image filenames belonging to 2 classes.
Found 1980 validated image filenames belonging to 2 classes.


In [119]:

cellType_train_data = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory='Image_classification_data/patch_images',
    x_col='ImageName', 
    y_col='cellTypeName',  
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
)


cellType_val_data = val_datagen.flow_from_dataframe(
    dataframe=val_data,
    directory='Image_classification_data/patch_images',
    x_col='ImageName',  
    y_col='cellTypeName',  
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
)

cellType_test_data = val_datagen.flow_from_dataframe(
    dataframe=test_data,
    directory='Image_classification_data/patch_images',
    x_col='ImageName',  
    y_col='cellTypeName',  
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
)

Found 5937 validated image filenames belonging to 4 classes.
Found 1979 validated image filenames belonging to 4 classes.
Found 1980 validated image filenames belonging to 4 classes.


In [120]:
isCancerousModel_base = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(27, 27, 3)), 
    Conv2D(32, (3, 3), activation='relu', padding='same'), 
    MaxPooling2D(2, 2),
            
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid') 
])

/home/leo/cancerAI/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [121]:
isCancerousModel_elu = Sequential([
    Conv2D(32, (3, 3), activation='elu', padding='same', input_shape=(27, 27, 3)), 
    Conv2D(32, (3, 3), activation='elu', padding='same'), 
    MaxPooling2D(2, 2),
            
    Conv2D(64, (3, 3), padding='same', activation='elu'),
    Conv2D(64, (3, 3), padding='same', activation='elu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(64, activation='elu'),
    Dense(1, activation='sigmoid') 
])

isCancerousModel_PReLU = Sequential([
    Conv2D(32, (3, 3), padding='same', input_shape=(27, 27, 3)),
    PReLU(alpha_initializer='zeros'),
    
    Conv2D(32, (3, 3), padding='same'),
    PReLU(alpha_initializer='zeros'),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), padding='same'),
    PReLU(alpha_initializer='zeros'),

    Conv2D(64, (3, 3), padding='same'),
    PReLU(alpha_initializer='zeros'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(64),
    PReLU(alpha_initializer='zeros'),
    
    Dense(1, activation='sigmoid')
])

isCancerousModel_swish = Sequential([
    Conv2D(32, (3, 3), activation=swish, padding='same', input_shape=(27, 27, 3)), 
    Conv2D(32, (3, 3), activation=swish, padding='same'), 
    MaxPooling2D(2, 2),
            
    Conv2D(64, (3, 3), padding='same', activation=swish),
    Conv2D(64, (3, 3), padding='same', activation=swish),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(64, activation=swish),
    Dense(1, activation='sigmoid') 
])


In [122]:
isCancerousModel_3Layers = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(27, 27, 3)), 
    Conv2D(32, (3, 3), activation='relu', padding='same'), 
    MaxPooling2D(2, 2),
            
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), padding='same', activation='relu'),
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid') 
])

isCancerousModel_simplifiedLayers = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(27, 27, 3)),
    MaxPooling2D(2, 2),
            
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid') 
])

isCancerousModel_dropout = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(27, 27, 3)), 
    Conv2D(32, (3, 3), activation='relu', padding='same'), 
    MaxPooling2D(2, 2),
            
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid') 
])

In [123]:
isCancerousModels_activation = {
    "ELU activation": isCancerousModel_elu,
    "PReLU activation": isCancerousModel_PReLU,
    "Swish activation": isCancerousModel_swish,
}


isCancerousModels_structure = {
    "3 Layer": isCancerousModel_3Layers,
    "Simplified layers": isCancerousModel_simplifiedLayers,
    "Dropout": isCancerousModel_dropout,
}


In [124]:
cellTypeModel_base = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(27, 27, 3)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2),
            
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(64, activation='relu'),
    Dense(4, activation='softmax') 
])



cellTypeModel_swish = Sequential([
    Conv2D(32, (3, 3), activation=swish, padding='same', input_shape=(27, 27, 3)),
    Conv2D(32, (3, 3), activation=swish, padding='same'),
    MaxPooling2D(2, 2),
            
    Conv2D(64, (3, 3), padding='same', activation=swish),
    Conv2D(64, (3, 3), padding='same', activation=swish),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(64, activation='swish'),
    Dense(4, activation='softmax') 
])

In [125]:
cellTypeModel_elu = Sequential([
    Conv2D(32, (3, 3), activation='elu', padding='same', input_shape=(27, 27, 3)),
    Conv2D(32, (3, 3), activation='elu', padding='same'),
    MaxPooling2D(2, 2),
            
    Conv2D(64, (3, 3), padding='same', activation='elu'),
    Conv2D(64, (3, 3), padding='same', activation='elu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(64, activation='elu'),
    Dense(4, activation='softmax') 
])

cellTypeModel_PReLU = Sequential([
    Conv2D(32, (3, 3), padding='same', input_shape=(27, 27, 3)),
    PReLU(alpha_initializer='zeros'),
    
    Conv2D(32, (3, 3), padding='same'),
    PReLU(alpha_initializer='zeros'),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), padding='same'),
    PReLU(alpha_initializer='zeros'),

    Conv2D(64, (3, 3), padding='same'),
    PReLU(alpha_initializer='zeros'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(64),
    PReLU(alpha_initializer='zeros'),
    
    Dense(4, activation='softmax')
])

In [126]:
cellTypeModel_3Layers = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(27, 27, 3)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2),
            
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), padding='same', activation='relu'),
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax') 
])

cellTypeModel_simplifiedLayers = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(27, 27, 3)),
    MaxPooling2D(2, 2),
            
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(64, activation='relu'),
    Dense(4, activation='softmax') 
])

cellTypeModel_dropout = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(27, 27, 3)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2),
            
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(4, activation='softmax') 
])

In [127]:
cellTypeModels_activation = {
    "ELU activation": cellTypeModel_elu,
    "PReLU activation": cellTypeModel_PReLU,
    "Swish activation": cellTypeModel_swish,
}

cellTypeModels_structure = {
    "3 Layer": cellTypeModel_3Layers,
    "Simplified layers": cellTypeModel_simplifiedLayers,
    "Dropout": cellTypeModel_dropout,
}

In [ ]:
results = []

def train_and_evaluate_model(name, model, train_data, val_data, test_data, loss, metrics, epochs, class_weight):
    print(f"\nTraining {name} model...\n")
    
    model.compile(optimizer='adam', loss=loss, metrics=metrics)

    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )

    start_time = time.time()
    history = model.fit(
        train_data,
        validation_data=val_data,
        epochs=epochs,
        callbacks=[early_stop],
        class_weight=class_weight,
        verbose=0
    )
    
    total_time = time.time() - start_time
    avg_epoch_time = total_time / len(history.history['loss'])
    num_epochs = len(history.history['loss'])

    losses = []
    accuracies = []

    for i in range(20):
        loss, acc = model.evaluate(test_data)
        losses.append(loss)
        accuracies.append(acc)

    avg_loss = np.mean(losses)
    avg_acc = np.mean(accuracies)

    y_true = val_data.classes
    y_pred = model.predict(val_data, verbose=0)
    
    if model.output_shape[-1] == 1:
        y_pred = (y_pred > 0.5).astype(int)
    else:
        y_pred = y_pred.argmax(axis=1)

    results.append({
        'Model': name,
        'Total Time (s)': round(total_time, 2),
        'Avg Epoch  Time (s)': round(avg_epoch_time, 2),
        'Total Epochs': num_epochs,
        'Accuracy': round(avg_acc, 4),
        'Loss': round(avg_loss, 4),
    })

    return model, history



In [129]:
trained_model, history = train_and_evaluate_model(
    name="is cancerous base",
    model=isCancerousModel_base,
    train_data=isCancerous_train_data,
    val_data=isCancerous_val_data,
    test_data=isCancerous_test_data,
    loss='binary_crossentropy',
    metrics=['accuracy'],
    epochs=50,
    class_weight=isCancerous_class_weights_dict
)

df_results = pd.DataFrame(results)
print(df_results)
results = []

for model_name, builder in isCancerousModels_activation.items():
    model = builder

    trained_model, history = train_and_evaluate_model(
        name=model_name,
        model=model,
        train_data=isCancerous_train_data,
        val_data=isCancerous_val_data,
        test_data=isCancerous_test_data,
        loss='binary_crossentropy',
        metrics=['accuracy'],
        epochs=50,
        class_weight=isCancerous_class_weights_dict
    )

df_results = pd.DataFrame(results)
df_results.to_csv("is_cancer_activation_test_results.csv", index=False)
print(df_results)
results = []

for model_name, builder in isCancerousModels_structure.items():
    model = builder

    trained_model, history = train_and_evaluate_model(
        name=model_name,
        model=model,
        train_data=isCancerous_train_data,
        val_data=isCancerous_val_data,
        test_data=isCancerous_test_data,
        loss='binary_crossentropy',
        metrics=['accuracy'],
        epochs=50,
        class_weight=isCancerous_class_weights_dict
    )

df_results = pd.DataFrame(results)
df_results.to_csv("is_cancer_structure_test_results.csv", index=False)
print(df_results)
results = []


Training is cancerous base model...



/home/leo/cancerAI/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


               Model  Total Time (s)  Avg Epoch  Time (s)  Total Epochs  \
0  is cancerous base           85.34                  6.1            14   

   Accuracy    Loss  
0    0.9106  0.2409  

Training ELU activation model...


Training PReLU activation model...


Training Swish activation model...

              Model  Total Time (s)  Avg Epoch  Time (s)  Total Epochs  \
0    ELU activation          105.12                 5.84            18   
1  PReLU activation           87.24                 6.23            14   
2  Swish activation          118.81                 5.94            20   

   Accuracy    Loss  
0    0.9086  0.2321  
1    0.9081  0.2414  
2    0.9126  0.2437  

Training 3 Layer model...


Training Simplified layers model...


Training Dropout model...

               Model  Total Time (s)  Avg Epoch  Time (s)  Total Epochs  \
0            3 Layer           88.11                 5.87            15   
1  Simplified layers          223.80                 5.60          

In [130]:

trained_model, history = train_and_evaluate_model(
    name="Cell type base",
    model=cellTypeModel_base,
    train_data=cellType_train_data,
    val_data=cellType_val_data,
    test_data=cellType_test_data,
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy'],
    epochs=50,
    class_weight=cellType_class_weights_dict
)

df_results = pd.DataFrame(results)
print(df_results)
results = []

for model_name, builder in cellTypeModels_activation.items():
    model = builder

    trained_model, history = train_and_evaluate_model(
        name=model_name,
        model=model,
        train_data=cellType_train_data,
        val_data=cellType_val_data,
        test_data=cellType_test_data,
        loss='categorical_crossentropy',
        metrics=['categorical_accuracy'],
        epochs=50,
        class_weight=cellType_class_weights_dict
    )

df_results = pd.DataFrame(results)
df_results.to_csv("cell_type_activation_test_results.csv", index=False)
print(df_results)
results = []

for model_name, builder in cellTypeModels_structure.items():
    model = builder

    trained_model, history = train_and_evaluate_model(
        name=model_name,
        model=model,
        train_data=cellType_train_data,
        val_data=cellType_val_data,
        test_data=cellType_test_data,
        loss='categorical_crossentropy',
        metrics=['categorical_accuracy'],
        epochs=50,
        class_weight=cellType_class_weights_dict
    )

df_results = pd.DataFrame(results)
df_results.to_csv("cell_type_structure_test_results.csv", index=False)
print(df_results)


Training Cell type base model...



/home/leo/cancerAI/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


            Model  Total Time (s)  Avg Epoch  Time (s)  Total Epochs  \
0  Cell type base          144.36                 5.77            25   

   Accuracy    Loss  
0    0.7773  0.5847  

Training ELU activation model...


Training PReLU activation model...


Training Swish activation model...

              Model  Total Time (s)  Avg Epoch  Time (s)  Total Epochs  \
0    ELU activation           90.44                 6.03            15   
1  PReLU activation          108.28                 6.02            18   
2  Swish activation           87.73                 5.85            15   

   Accuracy    Loss  
0    0.7606  0.6427  
1    0.7722  0.6039  
2    0.7465  0.6574  

Training 3 Layer model...


Training Simplified layers model...


Training Dropout model...

               Model  Total Time (s)  Avg Epoch  Time (s)  Total Epochs  \
0            3 Layer          111.44                 5.87            19   
1  Simplified layers          112.34                 5.62            20  